In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://www.investopedia.com/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
}

In [3]:
response = requests.get(url, headers=headers)

In [4]:
soup = BeautifulSoup(response.text, "html.parser")

In [87]:
url = {}

In [26]:
list_items = soup.find_all("li", class_="header-nav__list-item")

In [92]:
for item in list_items:
    key = item.a.text.strip()
    links = item.find_all('a')
    value = list(set(link['href'] for link in links))
    url[key] = value

url.pop("Simulator")
url.pop("Academy")

['https://academy.investopedia.com/products/investing-for-beginners',
 'https://academy.investopedia.com/collections/investing-courses-catalog',
 'https://academy.investopedia.com/products/technical-analysis',
 'https://academy.investopedia.com/products/trading-for-beginners',
 'https://academy.investopedia.com/collections/trading-courses-collection',
 'https://academy.investopedia.com/',
 'https://academy.investopedia.com/products/become-a-day-trader']

In [93]:
url

{'Investing': ['https://www.investopedia.com/etfs-4427784',
  'https://www.investopedia.com/cryptocurrency-4427699',
  'https://www.investopedia.com/bonds-4689778',
  'https://www.investopedia.com/fundamental-analysis-4689757',
  'https://www.investopedia.com/options-and-derivatives-trading-4689663',
  'https://www.investopedia.com/financial-technology-and-automated-investing-4689759',
  'https://www.investopedia.com/trading-4427765',
  'https://www.investopedia.com/brokers-4689772',
  'https://www.investopedia.com/stocks-4427785',
  'https://www.investopedia.com/markets/',
  'https://www.investopedia.com/commodities-4427780',
  'https://www.investopedia.com/investing-4427685'],
 'Banking': ['https://www.investopedia.com/checking-accounts-4689732',
  'https://www.investopedia.com/savings-accounts-4689728',
  'https://www.investopedia.com/money-market-account-4689730',
  'https://www.investopedia.com/certificate-of-deposits-4689733',
  'https://www.investopedia.com/banking-4427754'],
 '

In [94]:
alph_items = soup.find_all("li", class_="comp terms-bar__item mntl-block")

In [98]:
for el in alph_items:
    key = el.a.text.strip()
    value = el.a['href']
    url[key] = value

In [99]:
url

{'Investing': ['https://www.investopedia.com/etfs-4427784',
  'https://www.investopedia.com/cryptocurrency-4427699',
  'https://www.investopedia.com/bonds-4689778',
  'https://www.investopedia.com/fundamental-analysis-4689757',
  'https://www.investopedia.com/options-and-derivatives-trading-4689663',
  'https://www.investopedia.com/financial-technology-and-automated-investing-4689759',
  'https://www.investopedia.com/trading-4427765',
  'https://www.investopedia.com/brokers-4689772',
  'https://www.investopedia.com/stocks-4427785',
  'https://www.investopedia.com/markets/',
  'https://www.investopedia.com/commodities-4427780',
  'https://www.investopedia.com/investing-4427685'],
 'Banking': ['https://www.investopedia.com/checking-accounts-4689732',
  'https://www.investopedia.com/savings-accounts-4689728',
  'https://www.investopedia.com/money-market-account-4689730',
  'https://www.investopedia.com/certificate-of-deposits-4689733',
  'https://www.investopedia.com/banking-4427754'],
 '

In [101]:
sum(len(value) for value in url.values())

1645